In [2]:
import requests

postcode_data_url = "http://www.nomisweb.co.uk/output/census/2011/Postcode_Estimates_Table_1.csv"

with requests.Session() as s:
    download = s.get(postcode_data_url)
    decoded_content = download.content.decode('utf-8')

csv_file = open("Postcode_Estimates_Table_1.csv", "w")
csv_file.write(decoded_content)
csv_file.close()



In [107]:
import pandas as pd

df_postcodes = pd.read_csv("Postcode_Estimates_Table_1.csv")
df_postcodes

,Postcode,Total,Males,Females,Occupied_Households
0,AL1 1AG,14,6,8,6
1,AL1 1AJ,124,60,64,51
2,AL1 1AR,32,17,15,17
3,AL1 1AS,34,17,17,13
4,AL1 1BH,52,15,37,41
...,...,...,...,...,...
1308775,YO8 9YA,23,14,9,8
1308776,YO8 9YB,33,17,16,13
1308777,YO8 9YD,9,4,5,4
1308778,YO8 9YE,13,6,7,3


# Data Cleaning

## Postcode Format

UK postcodes can be in inconsistent formats so we should first do some work to check out how they are formatted.

First we want to explore the format of the postcode strings. A good first check is to see if they are all the same length:

In [108]:
#check the lengths of the postcode strings
df_postcodes["Postcode"].apply(len).value_counts()


7    1308780
Name: Postcode, dtype: int64

They are all of length 7, that is a good start.

We'll use regex with capture groups to grab the different parts of the postcode. Postcodes can be in one of 6 formats:

| ![UK Postcode Formats](uk_postcode_format.png) |
|:--:|
| Obtained from https://ideal-postcodes.co.uk/guides/uk-postcode-format |

Inspecting the postcodes in the data set they are formated as:
 - Fixed length of 7 characters.
 - Outcodes left aligned.
 - Incodes right aligned.
 - Spaces in between the outcodes and incodes where they are not 7 characters long.

We will use regex with match groups to extract some of the postcode parts:
- Outward Code
- Inward Code
- Area
- Sector

District and sub-district duplicate information in outward code. Unit does not give us any more information than the full postcode.


In [145]:
# We can get the out, in and area in one, but the sector code overlaps the other match groups so we need to do this seperately

#regex to extract outward (1), area (2) and inner (3)
regex_str = r"^(([A-Z]{1,2})[0-9]{1,2}[A-Z]?)\s*([0-9][A-Z]{2})$"

df_postcodes[["Outward Code", "Area", "Inward Code"]] = df_postcodes["Postcode"].str.extract(r'^(([A-Z]{1,2})[0-9]{1,2}[A-Z]?)\s*([0-9][A-Z]{2})')

#sector is outward code plus the first digit of inner code
df_postcodes["Sector"] = df_postcodes["Outward Code"] + df_postcodes["Inward Code"].str[0]

df_postcodes


,Postcode,Total,Males,Females,Occupied_Households,Outward Code,Area,Inward Code,Sector
0,AL1 1AG,14,6,8,6,AL1,AL,1AG,AL11
1,AL1 1AJ,124,60,64,51,AL1,AL,1AJ,AL11
2,AL1 1AR,32,17,15,17,AL1,AL,1AR,AL11
3,AL1 1AS,34,17,17,13,AL1,AL,1AS,AL11
4,AL1 1BH,52,15,37,41,AL1,AL,1BH,AL11
...,...,...,...,...,...,...,...,...,...
1308775,YO8 9YA,23,14,9,8,YO8,YO,9YA,YO89
1308776,YO8 9YB,33,17,16,13,YO8,YO,9YB,YO89
1308777,YO8 9YD,9,4,5,4,YO8,YO,9YD,YO89
1308778,YO8 9YE,13,6,7,3,YO8,YO,9YE,YO89


Check that we have an expected range of lengths for each code, and that we have no nulls:

In [150]:
for col in ["Outward Code", "Area", "Inward Code", "Sector"]:
    print(col)
    print(df_postcodes[col].apply(len).value_counts())
    print("Nulls: " + str(df_postcodes[col].isnull().sum()))
    print()

Outward Code
3    644619
4    631294
2     32867
Name: Outward Code, dtype: int64
Nulls: 0

Area
2    1162930
1     145850
Name: Area, dtype: int64
Nulls: 0

Inward Code
3    1308780
Name: Inward Code, dtype: int64
Nulls: 0

Sector
4    644619
5    631294
3     32867
Name: Sector, dtype: int64
Nulls: 0

